Connect to Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/ANNDL Challenge2

# Summary
In this notebook is shown how we built a model based on a combination of one-dimensional convolutional layers and bidirectional LSTMs.

Import libraries

In [ ]:
# Fix randomness and hide warnings
seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)


import matplotlib.pyplot as plt


In [ ]:
# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

Resources Paths

In [ ]:
trainDataFile = "./fullTrainData.npy"
validationDataFile = "./fullValidationData.npy"

Variables initialization

In [ ]:
trainData = np.load(trainDataFile, allow_pickle=True)
validationData = np.load(validationDataFile, allow_pickle=True)

# For each set we consider only the original series
labels = {'originalSeries': 0}

### Manipulating the input dataset

In [ ]:
def build_sequences(series, window=300, stride=50, telescope=9):
  '''
  Split the single 'series' in multiple blocks of length 'window'. Each block is composed of a x part of length 'window' - 'telescope' and a y part of length 'telescope'.
  'Data
  'Window' is the length of the input of our network
  'Stride' is the number of samples to skip before starting the next window
  'Telescope' is the length of the output of our network
  '''
  blocks = []
  labels = []
  idx = 0

  # We divide a time series in multiple blocks
  # If the series length is not a multiple of the window size, then the remaining slice of the series is skipped
  while(idx + window <= len(series)):
    blocks.append(series[idx : (idx + window - telescope)])
    labels.append(series[(idx + window - telescope) : (idx + window)])
    idx += stride

  blocks = np.array(blocks)
  labels = np.array(labels)
  return np.array(blocks), np.array(labels)

In [ ]:
def compute_sequences_for_dataset(data, window, stride, telescope, minimum_length):
  '''
  Using the build_sequences function to increase the number of samples.
  'Data' is a list containing the starting series.
  'Window' is the length of the input of our network plus the telescope
  'Stride' is the number of samples to skip before starting the next window
  'Telescope' is the length of the output of our network
  'Minimum_length' is the minimum length that a series must have to be considered
  '''

  x = []
  y = []

  # Compute the various sequences for each stationary series in data
  for series in data:

    # We skip the series with length less than the chosen 'minimum_length'
    if (len(series) >= minimum_length):

      # If the series length is less than the window size we need to pad the series
      if (len(series) < window):
        padding_len = window - len(series)

        # We isolate the Y portion (the telescope part) to let us adding the padding at the end of the X portion
        temp = series[ len(series) - telescope : len(series)]

        # We isolate the X portion
        series  = series[0 : len(series) - telescope]

        # We create the padding as a series full of 2-s
        padding = np.full((padding_len), 2, dtype= 'float32')

        # Our resulting series is composed of the X portion, the padding and the Y portion
        series = np.concatenate((series, padding, temp))

      # the X portions of the blocks and the Y portions of the blocks are computed
      x_new, y_new = build_sequences(series, window, stride, telescope)

      for elem in x_new:
        x.append(elem)
      for elem in y_new:
        y.append(elem)

  x = np.array(x)
  y = np.array(y)

  return x, y

In [ ]:
# Define common variables

window = 218
telescope = 18
stride = 10

Using the build_sequences function to increase the number of samples in the training set

In [ ]:
# Using the build_sequences function to increase the number of samples in the training set

train_x = []
train_y = []
# In this case we consider only blocks with size equal to the window size without using any padding techniques. The time series (or the remaining slices) shorter than the window size are skipped.
train_x, train_y = compute_sequences_for_dataset(trainData[labels['originalSeries']], window, stride, telescope, window)
train_x = np.expand_dims(train_x, axis= 2)

print(train_x.shape)
print(train_y.shape)
print(f"Original number of sequences: {len(trainData[labels['originalSeries']])}")
print(f"Number of total sequences: {len(train_x)}")
print(f'By choosing a window equal to {window} and stride equal to {stride}, there are {len(train_x) - len(trainData[labels["originalSeries"]])} more time series')

Using the build_sequences function to increase the number of samples in the validation set

In [ ]:
# Using the build_sequences function to increase the number of samples in the validation set

val_x = []
val_y = []
# In this case we consider only blocks with size equal to the window size without using any padding techniques. The time series (or the remaining slices) shorter than the window size are skipped.
val_x, val_y = compute_sequences_for_dataset(validationData[labels['originalSeries']], window, window, telescope, window)
val_x = np.expand_dims(val_x, axis= 2)

print(val_x.shape)
print(val_y.shape)
print(f"Original number of sequences: {len(validationData[labels['originalSeries']])}")
print(f"Number of total sequences: {len(val_x)}")
print(f'By choosing a window equal to {window} and a stride equal to {stride}, there are {len(val_x) - len(validationData[labels["originalSeries"]])} more time series')

### Developing the network

In [ ]:
# Define common variables
input_shape = [window - telescope, 1]
output_shape = [telescope, 1]
batch_size = 64
epochs = 200

save_the_model_on_file = False                                                  # Flag that says if it should save the model on file
model_file = "./tmp/model"                                                      # The model file path

Define the network

In [ ]:
def build_model(input_shape, output_shape):

    input_layer = tfkl.Input(shape=input_shape)
    x = tfkl.Bidirectional(tfkl.LSTM(256, return_sequences=True, name='lstm'), name='bidirectional_lstm_1')(input_layer)

    x = tfkl.Conv1D(128, 3, padding='same', activation='relu', name='conv1')(x)
    x = tfkl.MaxPooling1D()(x)
    x = tfkl.Bidirectional(tfkl.LSTM(128, return_sequences=True, name='lstm'), name='bidirectional_lstm_2')(x)

    x = tfkl.Conv1D(128, 3, padding='same', activation='relu', name='conv2')(x)
    x = tfkl.MaxPooling1D()(x)
    x = tfkl.Bidirectional(tfkl.LSTM(128, return_sequences=True, name='lstm'), name='bidirectional_lstm_3')(x)

    x = tfkl.Conv1D(128, 3, padding='same', activation='relu', name='conv3')(x)
    x = tfkl.MaxPooling1D()(x)
    x = tfkl.Bidirectional(tfkl.LSTM(128, return_sequences=True, name='lstm'), name='bidirectional_lstm_4')(x)

    output_layer = tfkl.Conv1D(output_shape[1], 3, padding='same', name='output_layer')(x)
    crop_size = output_layer.shape[1] - output_shape[0]
    output_layer = tfkl.Cropping1D((0, crop_size), name='cropping')(output_layer)

    model = tf.keras.Model(inputs=input_layer, outputs=output_layer, name='CONV_LSTM_model')
    model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam())

    return model

Train the network

In [ ]:
# Create the model
model = build_model(input_shape, output_shape)
model.summary()

# Train the model
history = model.fit(
    x = train_x,
    y = train_y,
    batch_size = batch_size,
    epochs = epochs,
    validation_data=(val_x, val_y),
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=3, factor=0.9, min_lr=1e-5)
    ]
).history

#Plot loss
best_epoch = np.argmin(history['val_loss'])
plt.figure(figsize=(17,4))
plt.plot(history['loss'], label='Training loss', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation loss', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Squared Error')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(18,3))
plt.plot(history['lr'], label='Learning Rate', alpha=.8, color='#ff7f0e')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.legend()
plt.grid(alpha=.3)
plt.show()


In [ ]:
if save_the_model_on_file:
  model.save(model_file)
  del model

  model = tfk.models.load_model(model_file)

### Inferences on predictions

Inference the predictions and compute the MSE and MAE

In [ ]:
predictions = model.predict(val_x)

# Calculate and print Mean Squared Error (MSE)
mean_squared_error = tfk.metrics.mean_squared_error(val_y.flatten(), predictions.flatten()).numpy()
print(f"Mean Squared Error: {mean_squared_error}")

# Calculate and print Mean Absolute Error (MAE)
mean_absolute_error = tfk.metrics.mean_absolute_error(val_y.flatten(), predictions.flatten()).numpy()
print(f"Mean Absolute Error: {mean_absolute_error}")

In [ ]:
def inspect_timeseries_predictions(X, Y, preds, num, telescope):
    '''
    Randomly plot a number 'num' of series composed by the known x portion and the true y portion and the predicted portion.
    'X' is a list containing all the available x portions.
    'Y' is a list containing all the available y portions (the true).
    'Preds' is a list containing all the predicted y portions (the predictions).
    'Num' is the number of series to plot.
    'Telescope' is the length of each y portion (it is valid also for the length of each predicted portion).
    '''

    figs, axs = plt.subplots(num, 1, sharex=True, figsize=(17,17))
    for i in range(0, num):
        idx=np.random.randint(0,len(X))
        axs[i].plot(np.arange(len(X[idx])), X[idx])
        axs[i].plot(np.arange(len(X[idx]), len(X[idx])+telescope), Y[idx], color='orange')
        axs[i].plot(np.arange(len(X[idx]), len(X[idx])+telescope), preds[idx], color='green')
        axs[i].set_ylim(-1,1)
    plt.show()

In [ ]:
# Plot some predictions
inspect_timeseries_predictions(val_x, val_y, predictions, 10, telescope)